In [5]:
import numpy as np
import random
import theano
from theano import tensor as T

In [9]:
# Load Data
filename = 'SPECTF.dat'
data = np.loadtxt(filename, delimiter=',')
X = data[:, 1:]
y = np.array([data[:, 0]]).T
n, d = X.shape

#divide data to train and test for cross validation
select = list(range(n))
random.shuffle(select)
select_train = np.array(select[0:int(9*n / 10)])
select_test = np.array(select[int(9*n / 10):n])

data_train = data[select_train, :]
data_test = data[select_test, :]
print('First two rows of data\n\n')
print(data_train[:2])

First two rows of data


[[  1.  77.  61.  62.  68.  62.  58.  72.  68.  77.  71.  76.  77.  72.
   75.  62.  57.  77.  74.  61.  58.  72.  76.  69.  68.  56.  53.  57.
   54.  69.  70.  73.  79.  65.  70.  66.  68.  67.  66.  71.  67.  60.
   60.  53.  57.]
 [  1.  75.  79.  65.  74.  63.  64.  74.  73.  73.  71.  76.  79.  70.
   69.  55.  64.  69.  71.  79.  76.  77.  78.  65.  70.  60.  60.  65.
   61.  70.  65.  74.  71.  62.  62.  69.  67.  72.  72.  72.  66.  61.
   64.  55.  54.]]


In [10]:
#divide data to labels and features
data_train_Y=data_train[:,0]
data_train_X=data_train[:,1:data_train.shape[1]]

data_test_Y=data_test[:,0]
data_test_X=data_test[:,1:data_test.shape[1]]
print("First two labels of data\n\n")
print(data_train_Y[:2])
print("\n\nFirst two feature-rows of data\n\n")
print(data_train_X[:2])

First two labels of data


[ 1.  1.]


First two feature-rows of data


[[ 77.  61.  62.  68.  62.  58.  72.  68.  77.  71.  76.  77.  72.  75.
   62.  57.  77.  74.  61.  58.  72.  76.  69.  68.  56.  53.  57.  54.
   69.  70.  73.  79.  65.  70.  66.  68.  67.  66.  71.  67.  60.  60.
   53.  57.]
 [ 75.  79.  65.  74.  63.  64.  74.  73.  73.  71.  76.  79.  70.  69.
   55.  64.  69.  71.  79.  76.  77.  78.  65.  70.  60.  60.  65.  61.
   70.  65.  74.  71.  62.  62.  69.  67.  72.  72.  72.  66.  61.  64.
   55.  54.]]


In [11]:
def gradient_descent(X, Y, step_size, max_steps):
    def Xnorm(a):  # a is an m*n matrix, return is nD vector where element 'i' is the average of all m elements in 'i'th column
        a = np.array(a)
        norms = np.zeros(a.shape[1])
        if len(a.shape) != 1:
            for i in range(a.shape[1]):
                norms[i] = np.sum([a[j, i] for j in range(a.shape[0])]) / a.shape[0]
        else:
            for i in range(a.shape[1]):
                norms[i] = a[i]
        return norms

    for i in range(len(Y)):
        if Y[i] == 0:
            Y[i] = -1
    X = np.array(X)
    Xnorms = Xnorm(X)

    a = T.vector()
    b = T.vector()
    c = T.scalar()

    loss = np.log(1 + np.exp(-c * (a.dot(b))))

    grad = theano.function([a, b, c], T.grad(loss, b))

    beta = np.zeros(X.shape[1])
    for s in range(max_steps):
        r = list(range(X.shape[0] - 1))
        random.shuffle(r)
        for f in r:
            beta[0] = beta[0] - grad(X[f], beta, Y[f])[0] / Xnorms[0]
            for i in range(1, X.shape[1]):
                beta[i] = beta[i] - (step_size) * grad(X[f], beta, Y[f])[i] / (Xnorms[i] * X.shape[0])

    return beta

In [13]:
beta=gradient_descent(data_train_X,data_train_Y, step_size=10, max_steps=20)

def accuracy_score(Y_true, Y_predict):
    error=0
    for i in range(len(Y_predict)):
        if Y_true[i] != Y_predict[i]:
            error = error + 1
    error= error*100/len(Y_predict)
    return 100-error

Y_log_pred=[]
for i in range(data_test_X.shape[0]):
    if data_test_X[i].T.dot(beta)>0:
        Y_log_pred.append(1)
    else:
        Y_log_pred.append(0)
log_accuracy=accuracy_score(data_test_Y, Y_log_pred)

print("Logistic Reg. Accuracy = ", log_accuracy)

('Logistic Reg. Accuracy = ', 75)
